# **SetUp**

In [8]:
!nvidia-smi

Fri Feb 20 05:50:29 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.15              Driver Version: 570.86.15      CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 NVL                Off |   00000000:3D:00.0 Off |                    0 |
| N/A   48C    P0             76W /  400W |       1MiB /  95830MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

# **ZO**

实验层级配置

L0 Baseline（默认）
- `BATCHDIFF_CKPT = -1`
- 按 `SAVE_STEPS` 固定频率保存**完整 checkpoint**

L1 Batch Differential Checkpoint
- `BATCHDIFF_CKPT = 0`：Incremental（从 base 开始累积所有 updates）
- `BATCHDIFF_CKPT = 1`：Pure Differential（只保存当前 step 的 update）
- `BATCHDIFF_CKPT = N (N >= 2)`：Batch Differential（每 `N` 步生成一次新的 full checkpoint）

L2 CPU Shadow
- `ENABLE_SHADOW = 1`
- 依赖：`BATCHDIFF_CKPT >= 0`
- CPU 端实时维护 **shadow model**

L3 即时恢复
- `INSTANT_RECOVER = 1`
- 依赖：`L2 + GPU_FAIL_STEP`
- 故障后立即恢复

GPU 故障注入（旁路）
- `GPU_FAIL_STEP = N`
- 在第 `N` 步模拟 GPU 故障
- 可单独使用

Resume / Replay
- `BATCHDIFF_RESUME`
- 从指定的 full checkpoint 恢复，自动扫描并重放后续的 differential checkpoints

In [9]:
# 简单staging
!MODEL=Qwen/Qwen3-1.7B \
    TRAIN_NAME=Test_staging_6 TASK=SST2 MODE=ft LR=1e-7 EPS=1e-3 \
    STEPS=1000 EVAL_STEPS=100 SAVE_STEPS=100 DO_EVAL=0 \
    bash /home/users/u0001609/NonStopZO2/example/mezo_runner/mezo.sh

========== Configuration ==========
TAG: mezo-ft-1e-7-1e-3-0
BS: 16, LR: 1e-7, EPS: 1e-3, SEED: 0
STEPS: 1000, EVAL_STEPS: 100, SAVE_STEPS: 100
MODE: ft, DO_EVAL: 0
--- Batch Differential Checkpoint ---
BATCHDIFF_CKPT: -1 (-1=disabled, 0=incremental, 1=pure diff, N>=2=batch diff)
ENABLE_SHADOW: 0
INSTANT_RECOVER: 0
GPU_FAIL_STEP: -1
BATCHDIFF_RESUME: 
Extra args:  --no_eval 


2026-02-20 06:12:58,513 - INFO - NumExpr defaulting to 16 threads.
2026-02-20 06:12:59,564 - INFO - PyTorch version 2.9.0 available.
[2026-02-20 06:13:11,530] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2026-02-20 06:13:17,455] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False
OurArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
batchdiff_ckpt=-1,
batchdiff_resume=,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloa

In [ ]:
# diff
!MODEL=Qwen/Qwen3-1.7B \
    TRAIN_NAME=Test_staging_7 TASK=SST2 MODE=ft LR=1e-7 EPS=1e-3 \
    STEPS=1000 EVAL_STEPS=100 SAVE_STEPS=100 DO_EVAL=0 \
    BATCHDIFF_CKPT=1 \
    bash /home/users/u0001609/NonStopZO2/example/mezo_runner/mezo.sh

========== Configuration ==========
TAG: mezo-ft-1e-7-1e-3-0
BS: 16, LR: 1e-7, EPS: 1e-3, SEED: 0
STEPS: 1000, EVAL_STEPS: 100, SAVE_STEPS: 100
MODE: ft, DO_EVAL: 0
--- Batch Differential Checkpoint ---
BATCHDIFF_CKPT: 1 (-1=disabled, 0=incremental, 1=pure diff, N>=2=batch diff)
ENABLE_SHADOW: 0
INSTANT_RECOVER: 0
GPU_FAIL_STEP: -1
BATCHDIFF_RESUME: 
Extra args:  --batchdiff_ckpt 1 --no_eval 


# **Resume**

In [ ]:
# 简单staging
!MODEL=Qwen/Qwen3-1.7B \
    TRAIN_NAME=Test_staging_2 TASK=SST2 MODE=ft LR=1e-3 EPS=1e-3 \
    STEPS=1000 EVAL_STEPS=100 SAVE_STEPS=100 DO_EVAL=0 \
    RESUME_CKPT="/lvs0/rccs-hpbdrt/minqiu/ZO_ckpt/Test_staging_1-SST2-Qwen3-1.7B-mezo-ft-1e-3-1e-3-0/checkpoint-700" \
    bash /home/users/u0001609/NonStopZO2/example/mezo_runner/mezo.sh

# **Read CKPT**

In [5]:
import torch
import json

# 读取 .pt 文件
pt_path = "/lvs0/rccs-hpbdrt/minqiu/ZO_ckpt/Test_staging_5-SST2-Qwen3-1.7B-mezo-ft-1e-6-1e-4-0/checkpoint-100/scheduler.pt"
# pt_path = "/lvs0/rccs-hpbdrt/minqiu/ZO_ckpt/Test_staging_6-SST2-Qwen3-1.7B-mezo-ft-1e-7-1e-3-0/checkpoint-200/batch_diff_meta.pt"
meta = torch.load(pt_path, map_location='cpu', weights_only=True)
print("=== scheduler.pt ===")
print(meta)

# # 读取 .json 文件
# json_path = "/lvs0/rccs-hpbdrt/minqiu/ZO_ckpt/Test_staging_4-SST2-Qwen3-1.7B-mezo-ft-1e-6-1e-4-0/checkpoint-200/zo_replay_history.json"
# with open(json_path, 'r') as f:
#     history = json.load(f)
# print("\n=== zo_replay_history.json ===")
# print(history)

=== scheduler.pt ===
{'base_lrs': [1e-06], 'last_epoch': 0, '_step_count': 1, '_is_initial': False, '_get_lr_called_within_step': False, '_last_lr': [1e-06], 'lr_lambdas': [None]}


# **Evaluate**

In [15]:
!python3 /home/users/u0001609/NonStopZO2/experiments/analyze_checkpoint_time.py \
    /home/users/u0001609/NonStopZO2/experiments/log_draft.log \
    -v    # 显示每个checkpoint的详细信息


ZOTrainer Full Checkpoint Time Analysis
Log file: /home/users/u0001609/NonStopZO2/experiments/log_draft.log
Total checkpoints:  1
Average time:       20.607s
Median time:        20.607s
Standard deviation: 0.000s
Min time:           20.607s
Max time:           20.607s
Total time spent:   20.607s

Individual checkpoint times:
------------------------------
  Step    100: 20.607s
